In [1]:
using JSON

#load OD pair-route incidence
odPairRoute = readall("od_pair_route_incidence_MA_ext.json");
odPairRoute = JSON.parse(odPairRoute);

#load link-route incidence
linkRoute = readall("link_route_incidence_MA_ext.json");
linkRoute = JSON.parse(linkRoute);

#load OD pair labels
odPairLabel = readall("od_pair_label_dict_MA_refined_ext.json");
odPairLabel = JSON.parse(odPairLabel);

odPairLabel_ = readall("od_pair_label_dict__MA_refined_ext.json");
odPairLabel_ = JSON.parse(odPairLabel_);

#load link labels
linkLabel = readall("link_label_dict_MA_ext.json");
linkLabel = JSON.parse(linkLabel);

linkLabel_ = readall("link_label_dict_MA__ext.json");
linkLabel_ = JSON.parse(linkLabel_);

#load node-link incidence
nodeLink = readall("node_link_incidence_MA_ext.json");
nodeLink = JSON.parse(nodeLink);

In [2]:
include("load_network_uni-class.jl")

load_ta_network (generic function with 2 methods)

In [3]:
ta_data = load_ta_network("East_Massachusetts_Apr_PM_ext");

In [4]:
capacity = ta_data.capacity;
free_flow_time = ta_data.free_flow_time;

In [5]:
############
#Read in the demand file
file = open("../data_traffic_assignment_uni-class/East_Massachusetts_trips_Apr_PM_ext.txt")
demands = Dict{(Int64,Int64), Float64}()
s = 0
for line in eachline(file)
    if contains(line, "Origin")
        s = int(split(line)[2])
    else
        pairs = split(line, ";")
        for pair in pairs
            if !contains(pair, "\n")
                pair_vals = split(pair, ":")
                t, demand = int(pair_vals[1]), float(pair_vals[2])
                demands[(s,t)] = demand 
            end
        end
    end
end                
close(file)

In [6]:
demands;

In [7]:
odPairLabel_;

In [8]:
demandsVec = zeros(462)

for i = 1:length(demandsVec)
    demandsVec[i] = demands[(odPairLabel_["$i"][1], odPairLabel_["$i"][2])]
end

In [9]:
demandsVec;

In [10]:
for key=keys(odPairRoute)
    if contains(key, "462-")
        println(key)
    end
end

462-329


In [11]:
linkRoute;

In [12]:
coeffs_dict_Apr_PM_ = readall("../temp_files/coeffs_dict_Apr_PM_ext.json")
coeffs_dict_Apr_PM_ = JSON.parse(coeffs_dict_Apr_PM_)
fcoeffs = coeffs_dict_Apr_PM_["(8,1.5,100.0,1)"]

# fcoeffs = [1, 0, 0, 0, 0, 0, 0, 0, 0]

9-element Array{Any,1}:
  1.0        
  0.0337656  
  0.0204459  
  0.00246414 
 -0.00619792 
 -0.00518046 
 -0.00234895 
 -0.000318631
 -7.77438e-5 

In [13]:
using JuMP

# m = Model(solver=GurobiSolver(OutputFlag=false))
m = Model()

numLinks = 74
numRoute = 462
numOD = 462

@defVar(m, linkFlow[1:numLinks])
@defVar(m, pathFlow[1:numRoute])

pathFlowSum = Dict()

for i=1:numOD
    pathFlowSum[i] = 0
    for j=1:numRoute
        if "$(i)-$(j)" in keys(odPairRoute)
            pathFlowSum[i] += pathFlow[j]
        end
    end
    @addConstraint(m, pathFlowSum[i] == demandsVec[i])
end

pathFlowLinkSum = Dict()

for a=1:numLinks
    pathFlowLinkSum[a] = 0
    for j=1:numRoute
        if "$(a)-$(j)" in keys(linkRoute)
            pathFlowLinkSum[a] += pathFlow[j]
        end
    end
    @addConstraint(m, pathFlowLinkSum[a] == linkFlow[a])
end

for j=1:numRoute
    @addConstraint(m, pathFlow[j] >= 0)
end

# @defNLExpr(f, sum{free_flow_time[a]*linkFlow[a] + .03*free_flow_time[a]*((linkFlow[a])^5)/((capacity[a])^4), a = 1:numLinks})

@defNLExpr(f, sum{free_flow_time[a] * fcoeffs[1] * linkFlow[a] + 
    free_flow_time[a] * fcoeffs[2] * linkFlow[a]^2 / capacity[a] +
    free_flow_time[a] * fcoeffs[3] * linkFlow[a]^3 / capacity[a]^2 +
    free_flow_time[a] * fcoeffs[4] * linkFlow[a]^4 / capacity[a]^3 +
    free_flow_time[a] * fcoeffs[5] * linkFlow[a]^5 / capacity[a]^4 +
    free_flow_time[a] * fcoeffs[6] * linkFlow[a]^6 / capacity[a]^5 +
    free_flow_time[a] * fcoeffs[7] * linkFlow[a]^7 / capacity[a]^6 +
    free_flow_time[a] * fcoeffs[8] * linkFlow[a]^8 / capacity[a]^7 +
    free_flow_time[a] * fcoeffs[9] * linkFlow[a]^9 / capacity[a]^8, a = 1:numLinks})

@setNLObjective(m, Min, f)

solve(m)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.1, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     2265
Number of nonzeros in inequality constraint Jacobian.:      462
Number of nonzeros in Lagrangian Hessian.............:       74

Total number of variables............................:      536
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equa

:Optimal

In [14]:
getValue(linkFlow)

74-element Array{Float64,1}:
 2356.88       
 2226.33       
 5304.68       
 5500.5        
  385.617      
 2732.43       
 6353.37       
 3095.88       
 1875.59       
 5248.71       
  962.42       
  977.469      
  149.637      
    ⋮          
 2713.32       
 2357.74       
 1874.32       
 2925.36       
    3.97964e-46
    3.33701e-37
   -6.84228e-48
    2.73691e-48
 1103.8        
  572.559      
 1254.8        
 1905.48       

In [15]:
getObjectiveValue(m)

10786.929259374823

In [16]:
flows = Dict{(Int64,Int64),Float64}()

for i = 1:length(ta_data.start_node)
    key = (ta_data.start_node[i], ta_data.end_node[i])
    flows[key] = getValue(linkFlow)[i]
end

flows

Dict{(Int64,Int64),Float64} with 74 entries:
  (17,15) => 572.5585639999999
  (3,6)   => 1875.593056
  (21,13) => 1197.057779
  (22,16) => 3.337007329389173e-37
  (19,12) => 1.8320739880346514e-36
  (22,14) => -6.842277657836021e-48
  (15,17) => 1103.803223
  (10,11) => 824.255244
  (13,12) => 0.0
  (3,1)   => 5500.498989
  (16,13) => 2357.741
  (4,5)   => 4235.155262
  (13,16) => 2713.316169
  (17,16) => 1905.475671
  (20,13) => 1060.399082
  (12,8)  => 1903.111711
  (2,4)   => 6353.366730999999
  (16,17) => 1254.7969999999998
  (1,2)   => 2356.876493
  (14,21) => 1197.057779
  (2,3)   => 385.616726
  (13,19) => 2.3947971802426073e-48
  (13,14) => 0.0
  (16,14) => 2925.3608579999996
  (8,12)  => 1115.473151
  ⋮       => ⋮

In [17]:
# getValue(linkFlow)

In [18]:
# getObjectiveValue(m)

In [19]:
using PyCall
unshift!(PyVector(pyimport("sys")["path"]), "");
@pyimport GLS_Apr_weekday_PM_ext

No dicts found; please check load_dicts...


In [20]:
flow_user = GLS_Apr_weekday_PM_ext.x_

74x30 Array{Float64,2}:
 1366.35    791.95    771.324  …  1015.01        1624.03     280.272
 1206.76    992.338   991.158     1000.59        1273.13     570.559
 2114.24   1499.68   1200.6       1171.78        1608.83     947.273
 2273.84   1299.3     980.768     1186.21        1959.73     656.986
   58.465   201.97    271.021       86.5748        40.0646   261.282
  139.064   247.675   201.826  …    23.5801       146.69     143.578
 1193.7    1571.81   1353.56       781.132       1109.21    1362.48 
  953.514  1726.49   1642.59       829.701        651.679   1770.47 
 2124.9    1398.79    938.436     1533.92        1629.54    1007.64 
 2365.09   1244.1     649.406     1485.35        2087.07     599.646
  406.406   514.047   567.011  …   469.666        455.737    655.187
  281.997  1677.2    1591.06       204.042        239.965   1660.44 
  204.522   246.959  1020.38         1.2451e-13   268.26    1131.08 
    ⋮                          ⋱                                    
  817.703 

In [21]:
function socialObj(linkFlowVec)
    objVal = sum([free_flow_time[a] * fcoeffs[1] * linkFlowVec[a] + free_flow_time[a] * fcoeffs[2] * linkFlowVec[a]^2 / capacity[a] + free_flow_time[a] * fcoeffs[3] * linkFlowVec[a]^3 / capacity[a]^2 + free_flow_time[a] * fcoeffs[4] * linkFlowVec[a]^4 / capacity[a]^3 + free_flow_time[a] * fcoeffs[5] * linkFlowVec[a]^5 / capacity[a]^4 + free_flow_time[a] * fcoeffs[6] * linkFlowVec[a]^6 / capacity[a]^5 + free_flow_time[a] * fcoeffs[7] * linkFlowVec[a]^7 / capacity[a]^6 + free_flow_time[a] * fcoeffs[8] * linkFlowVec[a]^8 / capacity[a]^7 + free_flow_time[a] * fcoeffs[9] * linkFlowVec[a]^9 / capacity[a]^8 for a = 1:numLinks])
    return objVal
end

socialObj (generic function with 1 method)

In [22]:
week_day_Apr_list = [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 30]

for i in week_day_Apr_list
    println(socialObj(flow_user[:, i])/getObjectiveValue(m))
end

1.6273665104006652
1.6869952920955673
1.726476026056497
1.8858964227019863
0.9207394618646391
1.5574521344520689
1.6320186948422795
1.7414695697616003
1.9684419287479908
1.754985656991627
1.3364657395067934
1.6700948515685867
1.415326583369284
1.2521811889221561
0.921153058316567
2.0322902534283442
1.6644202442440597
1.6714983719751098
1.6428488779936092
1.2641648791422468
1.6242762092783825
